In [40]:
import os.path
import re
import hashlib
import tensorflow as tf
from tensorflow import gfile
from tensorflow import compat
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
import cv2
import numpy as np

In [2]:
"""
I took this function from 'https://github.com/loicmarie/sign-language-alphabet-recognizer', check it out!
I also took the dataset, although I splitted it in a different way.
"""

MAX_NUM_IMAGES_PER_CLASS = 2 ** 27 - 1  # ~134M

def create_image_lists(image_dir, testing_percentage, validation_percentage):
    """
    Brief:
        Builds a list of training images from the file system.
        Analyzes the sub folders in the image directory, splits them into stable
        training, testing, and validation sets, and returns a data structure
        describing the lists of images for each label and their paths.
    Args:
        image_dir: String path to a folder containing subfolders of images.
        testing_percentage: Integer percentage of the images to reserve for tests.
        validation_percentage: Integer percentage of images reserved for validation.
    Returns:
        A dictionary containing an entry for each label subfolder, with images split
        into training, testing, and validation sets within each label.
    """
    if not gfile.Exists(image_dir):
        print("Image directory '" + image_dir + "' not found.")
        return None
    result = {}
    sub_dirs = [x[0] for x in gfile.Walk(image_dir)]
    # The root directory comes first, so skip it.
    is_root_dir = True
    for sub_dir in sub_dirs:
        if is_root_dir:
            is_root_dir = False
            continue
        extensions = ['jpg', 'jpeg', 'JPG', 'JPEG']
        file_list = []
        dir_name = os.path.basename(sub_dir)
        if dir_name == image_dir:
            continue
        print("Looking for images in '" + dir_name + "'")
        for extension in extensions:
            file_glob = os.path.join(image_dir, dir_name, '*.' + extension)
            file_list.extend(gfile.Glob(file_glob))
        if not file_list:
            print('No files found')
            continue
        if len(file_list) < 20:
            print('WARNING: Folder has less than 20 images, which may cause issues.')
        elif len(file_list) > MAX_NUM_IMAGES_PER_CLASS:
            print('WARNING: Folder {} has more than {} images. Some images will '
                  'never be selected.'.format(dir_name, MAX_NUM_IMAGES_PER_CLASS))
        label_name = re.sub(r'[^a-z0-9]+', ' ', dir_name.lower())
        training_images = []
        testing_images = []
        validation_images = []
        for file_name in file_list:
            base_name = os.path.basename(file_name)
            # We want to ignore anything after '_nohash_' in the file name when
            # deciding which set to put an image in, the data set creator has a way of
            # grouping photos that are close variations of each other. For example
            # this is used in the plant disease data set to group multiple pictures of
            # the same leaf.
            hash_name = re.sub(r'_nohash_.*$', '', file_name)
            # This looks a bit magical, but we need to decide whether this file should
            # go into the training, testing, or validation sets, and we want to keep
            # existing files in the same set even if more files are subsequently
            # added.
            # To do that, we need a stable way of deciding based on just the file name
            # itself, so we do a hash of that and then use that to generate a
            # probability value that we use to assign it.
            hash_name_hashed = hashlib.sha1(compat.as_bytes(hash_name)).hexdigest()
            percentage_hash = ((int(hash_name_hashed, 16) %
                                (MAX_NUM_IMAGES_PER_CLASS + 1)) *
                             (100.0 / MAX_NUM_IMAGES_PER_CLASS))
            if percentage_hash < validation_percentage:
                validation_images.append(base_name)
            elif percentage_hash < (testing_percentage + validation_percentage):
                testing_images.append(base_name)
            else:
                training_images.append(base_name)
        result[label_name] = {
            'dir': dir_name,
            'training': training_images,
            'testing': testing_images,
            'validation': validation_images,
            }
    return result

In [3]:
result = create_image_lists("dataset", 20, 20)

Looking for images in 'A'
Looking for images in 'B'
Looking for images in 'C'
Looking for images in 'D'
Looking for images in 'del'
Looking for images in 'E'
Looking for images in 'F'
Looking for images in 'G'
Looking for images in 'H'
Looking for images in 'I'
Looking for images in 'J'
Looking for images in 'K'
Looking for images in 'L'
Looking for images in 'M'
Looking for images in 'N'
Looking for images in 'nothing'
Looking for images in 'O'
Looking for images in 'P'
Looking for images in 'Q'
Looking for images in 'R'
Looking for images in 'S'
Looking for images in 'space'
Looking for images in 'T'
Looking for images in 'U'
Looking for images in 'V'
Looking for images in 'W'
Looking for images in 'X'
Looking for images in 'Y'
Looking for images in 'Z'


In [ ]:
# Let's check how we can access the data
result['a']['training']

In [5]:
# This is the number of classes in which we have to classify our images (in this case 29, the letters of the alphabet and
# some other signs).
class_count = len(result.keys())

In [6]:
print(class_count)

29


In [ ]:
"""
Here I take the already splitted data returned by the function 'create_image_lists()' and I put each image path into 
two different lists, according to the related sign.
"""
train_data = []
validation_data = []

for val in result.keys():
    train_data.append(result[val]['training'])
    
for val in result.keys():
    validation_data.append(result[val]['validation'])

print(train_data)
print(validation_data)

In [30]:
"""
I wrote this two 'for' cycles to split the data into two folders --> 'train_data' and 'validation_data', which I'll use for the
training phase.
"""

for i in range(len(train_data)):
    for filename in train_data[i]:
        path = "dataset/" + list(result.keys())[i].upper() + "/" + filename 
        new_path = "train_data/" + list(result.keys())[i] + "/" + filename
        img = cv2.imread(path, 0)
        cv2.imwrite(new_path, img)
        
for i in range(len(validation_data)):
    for filename in validation_data[i]:
        path = "dataset/" + list(result.keys())[i].upper() + "/" + filename 
        new_path = "validation_data/" + list(result.keys())[i] + "/" + filename
        img = cv2.imread(path, 0)
        cv2.imwrite(new_path, img)

In [91]:
"""
This is the core of the project, the CNN architecture.
Nothing special, actually. Until now, I haven't tried other combinations.
"""
def create_model():
    K.set_image_dim_ordering('tf')

    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=(200, 200, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(29))
    model.add(Activation('softmax'))
    
    return model

In [92]:
model = create_model()

batch_size = 64

# This is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        horizontal_flip=True)

# This is the augmentation configuration we will use for testing:
test_datagen = ImageDataGenerator(rescale=1./255)

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

train_generator = train_datagen.flow_from_directory(
        'train_data', 
        target_size=(200, 200),
        batch_size=batch_size,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        'validation_data',
        target_size=(200, 200),
        batch_size=batch_size,
        class_mode='categorical')

Found 52252 images belonging to 29 classes.
Found 17309 images belonging to 29 classes.


In [ ]:
# Let's start training!
model.fit_generator(
        train_generator,
        steps_per_epoch=2000 // batch_size,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=800 // batch_size)
model.save_weights('first_try.h5')  # Always save your weights after training or during training

"""I used 'FloydHub' to train the network, reaching an accuracy of about 80% with 50 epochs."""

In [120]:
def load_trained_model(weights_path):
    final_model = create_model()
    final_model.load_weights(weights_path)
    return final_model

In [121]:
# Once trained, let's use the optimized weights to create a new model and test it out!
load_model = load_trained_model("first_try.h5")
load_model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [124]:
# Here I simply put some images into the directory 'test_data' and predict the result
for i in range(20):
    img = cv2.imread('test_data/F' + str(i+1) + '.jpg')
    if (img is not None):
        img = np.expand_dims(img, axis=0)
        result_class = load_model.predict_classes(img)
        print(list(result.keys())[result_class[0]].upper())

F
F
F
F
F
F
